In [1]:
import cv2
import matplotlib.pyplot as plt
import cvlib as cv
from cvlib.object_detection import draw_bbox

import numpy as np
from sklearn.cluster import KMeans
from collections import Counter
import json


# 1
def load_image(file_path):
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img


# 2
def detect_person(img):
    bbox, label, conf = cv.detect_common_objects(img)
    
    if label.count('person') > 1:   # Exceptional case : 2 or more persons are detected
        print('>>> Warning : More than 1 person is detected from the given picture!')
        idx = label.index('person')
        bbox = [bbox[idx]]
        label = [label[idx]]
        conf = [conf[idx]]
    elif label.count('person') == 1:   # Ideal case : only one main preson is detected
        idx = label.index('person')
        bbox = [bbox[idx]]
        label = [label[idx]]
        conf = [conf[idx]]
    else:   # Exceptional case : No person is detected
        print('>>> Warning : No person is detected from the given picture!')
        output_image = draw_bbox(img, bbox, label, conf)
        plt.imshow(output_image)
        plt.show()
    return bbox, label, conf




# 1. Load image
file_path = '../data/pictures_thecatwilliams/thecatwilliams_004_502Likes.png'
load_image(file_path)

# 2. Detect the object 'person' from image
bbox, label, conf = detect_person(img)

Using TensorFlow backend.


In [4]:
a = [1,2,3]
a.count(3)

1